In [57]:
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn

mapping = {'verified':0,'reviewTime':1,'reviewerID':2,'asin':3,"reviewText":4,"summary":5,"unixReviewTime":6,"sentiment":7,"id":8}
ls = []

for line in open('Data/music_reviews_train.json'):
    review_data = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data:
        if key in mapping:
            tmp[mapping[key]] = review_data[key]
    ls.append(tmp)
#df = pd.read_json('Data/music_reviews_train.json', typ='series', orient='records')

In [58]:
df = pd.DataFrame(ls)
df = df.drop(df[df[4].isna()].index) #Drops all None values for reviewtext

In [59]:
mp = {value:key for key, value in mapping.items()}
df = df.rename(columns=mp)

In [65]:
import torch
PAD = "<PAD>"
window_size=2
vocab = {"<PAD>": 0}
id = 1
word2idx = []

for line in df["reviewText"]:
    for word in line.split():
        if word not in vocab:
            vocab[word] = id
            id += 1

target = []

for line in df["reviewText"]:
    paddedline = [PAD, PAD] + line.split() + [PAD, PAD]
    for i in range(2, len(paddedline)-2):
        tmp = []
        target.append(vocab[paddedline[i]])
        for j in [-2,-1,1,2]:
            if i+j < len(paddedline):
                tmp.append(vocab[paddedline[i+j]])
        word2idx.append(tmp)


word2idx = torch.tensor(word2idx)
target = torch.tensor(target)

In [68]:
import torch.nn as nn

embed_dim = 64
vocab_dim = len(vocab)

class CBOW(nn.Module):
    def __init__(self, emb_dim, vocab_dim):
        super(CBOW, self).__init__()
        self.emb = nn.Embedding(vocab_dim, emb_dim)
        self.hidden = nn.Linear(embed_dim,vocab_dim)
        #self.loss = nn.CrossEntropyLoss()


    def forward(self, inputs, gold):
        self.x = self.emb(inputs)
        self.x2 = self.hidden(torch.sum(self.x,dim=0))
        #self.output = self.loss(self.x2, gold)
        return self.x2

CBOW = CBOW(embed_dim,len(vocab))
print(CBOW)

CBOW(
  (emb): Embedding(226348, 64)
  (hidden): Linear(in_features=64, out_features=226348, bias=True)
)


In [70]:
len(vocab)

226348

In [69]:
import torch.optim as optim
optimizer = optim.SGD(CBOW.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(zip(word2idx, target)):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        labels2 = labels
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        labels2 = labels.reshape(1)
        outputs = CBOW(inputs, labels2)
        outputs2 = outputs.view(-1,1)
        loss = criterion(outputs2.reshape(1,len(vocab)), labels2)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i <= 10:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss))
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: shape '[1, 14532]' is invalid for input of size 226348